In [75]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat

def Ajacob_at(x_esti):
    dt = 1
    A = np.eye(4) + dt *np.array([[0, 1, 0, 0],
                               [0, 0, 0, 0],
                               [0, 0, 0, 1],
                               [0, 0, 0, 0]])
    return A

def Hjacob_at(x_pred, S):
    H = np.zeros((4,4))
    H[0][0] = (x_pred[0]-S[0][0]) / np.sqrt((x_pred[0]-S[0][0])**2 + (x_pred[2]-S[0][1])**2)
    H[0][2] = (x_pred[2]-S[0][1]) / np.sqrt((x_pred[0]-S[0][0])**2 + (x_pred[2]-S[0][1])**2)
    H[1][0] = (x_pred[0]-S[1][0]) / np.sqrt((x_pred[0]-S[1][0])**2 + (x_pred[2]-S[1][1])**2)
    H[1][2] = (x_pred[2]-S[1][1]) / np.sqrt((x_pred[0]-S[1][0])**2 + (x_pred[2]-S[1][1])**2)
    H[2][0] = (x_pred[0]-S[2][0]) / np.sqrt((x_pred[0]-S[2][0])**2 + (x_pred[2]-S[2][1])**2)
    H[2][2] = (x_pred[2]-S[2][1]) / np.sqrt((x_pred[0]-S[2][0])**2 + (x_pred[2]-S[2][1])**2)
    H[3][0] = (x_pred[0]-S[3][0]) / np.sqrt((x_pred[0]-S[3][0])**2 + (x_pred[2]-S[3][1])**2)
    H[3][2] = (x_pred[2]-S[3][1]) / np.sqrt((x_pred[0]-S[3][0])**2 + (x_pred[2]-S[3][1])**2)
    return H

def fx(x_esti):
    dt = 1
    A = np.eye(4) + dt *np.array([[0, 1, 0, 0],
                               [0, 0, 0, 0],
                               [0, 0, 0, 1],
                               [0, 0, 0, 0]])
    return A * x_esti.T

def hx(x_pred, S):
    h = np.zeros((4,1))
    h[0][0] = np.sqrt((x_pred[0] - S[0][0])**2+(x_pred[2] - S[0][1])**2) 
    h[1][0] = np.sqrt((x_pred[0] - S[1][0])**2+(x_pred[2] - S[1][1])**2)
    h[2][0] = np.sqrt((x_pred[0] - S[2][0])**2+(x_pred[2] - S[2][1])**2)
    h[3][0] = np.sqrt((x_pred[0] - S[3][0])**2+(x_pred[2] - S[3][1])**2)
    return h

def extended_kalman_filter(z_meas, x_esti, P, S):
    """Extended Kalman Filter Algorithm."""
    # (1) Prediction.
    A = Ajacob_at(x_esti)
    x_pred = fx(x_esti)
    P_pred = A @ P @ A.T + Q

    # (2) Kalman Gain.
    H = Hjacob_at(x_pred,S)
    print(H)
    K = P_pred @ H.T @ inv(H @ P_pred @ H.T + R)

    # (3) Estimation.
    x_esti = x_pred + K @ (z_meas.T - hx(x_pred,S))

    # (4) Error Covariance.
    P = P_pred - K @ H @ P_pred
    
    print(x_esti)
    return x_esti.T, P

dt = 1
H = np.zeros((1, 4))
Q = np.eye(4)
R = np.eye(4)

x_0 = np.matrix([[1, 0, 1, 0]])  
#x_esti = np.matrix([[1, 0, 1, 0]]) 
print(x_0.item(0))
P_0 = 10 * np.eye(4)


xpos_pred = 0
ypos_pred = 0
H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])

for i in range(0,100):
    z_meas = np.matrix([4.8,2.41,8.16088231,7.161592])
    #print(x_0)
    if i == 0:
        x_esti, P = x_0, P_0
    else:
        x_esti, P = extended_kalman_filter(z_meas, x_esti, P, H)
        x_hat = [x_esti.item(0), x_esti.item(2)]
        print(x_hat)









1
[[-0.98851107  0.          0.15114848  0.        ]
 [-0.72516552  0.         -0.6885746   0.        ]
 [-0.99711665  0.          0.07588407  0.        ]
 [-0.89415483  0.         -0.4477579   0.        ]]
[[5.30777964]
 [2.05132364]
 [6.81020108]
 [2.76676242]]
[5.307779644595639, 6.810201075128967]
[[-0.01888537  0.          0.99982166  0.        ]
 [-0.07620342  0.          0.99709229  0.        ]
 [-0.57785925  0.          0.81613644  0.        ]
 [-0.95926656  0.          0.2825025   0.        ]]
[[7.21015944]
 [1.84176358]
 [6.83936447]
 [0.88197971]]
[7.210159443394409, 6.839364474334101]
[[ 0.19216159  0.          0.9813633   0.        ]
 [ 0.94728893  0.          0.32038052  0.        ]
 [-0.55023866  0.          0.83500743  0.        ]
 [-0.99961437  0.          0.02776878  0.        ]]
[[ 8.49926439]
 [ 1.58279981]
 [ 5.89419022]
 [-0.27545423]]
[8.499264393689925, 5.894190220068509]
[[ 0.4122022   0.          0.91109239  0.        ]
 [ 0.84762736  0.         -0.53059199  0

[[-2.83835178e-10  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-5.65314877e-10  0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [-8.08736084e-01  0.00000000e+00  5.88171698e-01  0.00000000e+00]
 [-9.21582799e-01  0.00000000e+00 -3.88181845e-01  0.00000000e+00]]
[[ 7.54000000e+00]
 [-3.68575495e-16]
 [ 4.80000000e+00]
 [ 3.98990266e-16]]
[7.53999999863759, 4.8000000013339355]
[[-2.83835363e-10  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-5.65315245e-10  0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [-8.08736084e-01  0.00000000e+00  5.88171698e-01  0.00000000e+00]
 [-9.21582799e-01  0.00000000e+00 -3.88181845e-01  0.00000000e+00]]
[[7.54000000e+00]
 [3.17962930e-17]
 [4.80000000e+00]
 [4.44814872e-16]]
[7.539999998637591, 4.8000000013339355]
[[-2.83835178e-10  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-5.65314877e-10  0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [-8.08736084e-01  0.00000000e+00  5.88171698e-01  0.00000000e+00]
 [-9.21582799e-01  0.00000000e+0

In [63]:
h = np.zeros((4,1))
print(h[0][0])

0.0
